In [1]:
import random
import pandas as pd
from transformers import BertTokenizer, BertModel, Trainer, TrainingArguments
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error

# 임의의 MIDI 데이터 생성 함수 (초 단위)
def generate_midi_data(seconds):
    data = []
    for i in range(seconds * 10):  # 초 x 10 (0.1초 간격)
        time = f"time_{i * 0.1:.1f}"
        notes = [f"note_{random.randint(40, 60)}_velocity_{random.randint(50, 70)}" for _ in range(random.randint(1, 3))]
        entry = f"{time}: " + ", ".join(notes)
        data.append(entry)
    return data  # 데이터 목록 반환

# 94초 MIDI 데이터 생성
midi_data_94s = generate_midi_data(94)

# 10초씩 나누기 (각 시퀀스는 100개의 요소, 즉 10초 데이터)
segment_length = 100
midi_segments = ["; ".join(midi_data_94s[i:i + segment_length]) for i in range(0, len(midi_data_94s), segment_length)]

# 마지막 시퀀스가 부족할 경우 패딩 추가
if len(midi_segments[-1].split('; ')) < segment_length:
    last_segment = midi_segments.pop()
    padding = "; ".join([f"time_{(len(last_segment.split('; ')) + i) * 0.1:.1f}: note_0_velocity_0" for i in range(segment_length - len(last_segment.split('; ')))])
    midi_segments.append(last_segment + "; " + padding)

# BERT tokenizer 초기화
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 점수 데이터 생성 및 정규화 (0에서 1 사이로)
scores = [[random.uniform(0, 100) for _ in range(3)] for _ in range(len(midi_segments))]
normalized_scores = [[score / 100 for score in s] for s in scores]  # 정규화

# 커스텀 BERT 모델 정의 (회귀를 위한 FFN 추가)
class BertForMultipleRegression(nn.Module):
    def __init__(self):
        super(BertForMultipleRegression, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.regressor = nn.Linear(self.bert.config.hidden_size, 3)  # 3개의 점수 예측

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        predictions = self.regressor(pooled_output)

        if labels is not None:
            loss_fn = nn.MSELoss()
            loss = loss_fn(predictions, labels)
            return loss, predictions
        return predictions

model = BertForMultipleRegression()

# PyTorch Dataset 클래스 정의
class MidiDataset(Dataset):
    def __init__(self, midi_segments, scores):
        self.midi_segments = midi_segments
        self.scores = scores

    def __len__(self):
        return len(self.midi_segments)

    def __getitem__(self, idx):
        midi_data = self.midi_segments[idx]
        score = self.scores[idx]
        inputs = tokenizer(midi_data, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
        item = {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "labels": torch.tensor(score, dtype=torch.float)  # 다중 점수 예측을 위한 레이블
        }
        return item

# Dataset 생성
train_dataset = MidiDataset(midi_segments, normalized_scores)

# MSE 손실 계산 함수
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions
    mse = mean_squared_error(labels, preds)
    return {"mse": mse}

# training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=50,
    per_device_train_batch_size=2,
    learning_rate=1e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

# 입력 데이터와 예측할 데이터 시각화
input_data = pd.DataFrame({
    'Segment': [f'Segment {i + 1}' for i in range(len(midi_segments))],
    'MIDI Data': midi_segments,
    'Scores': scores  # 원래 점수 표시
})

print("\nInput Data and Predicted Scores:")
print(input_data)

# 예측 결과 출력 (역정규화 포함)
for i, data in enumerate(midi_segments, start=1):
    inputs = tokenizer(data, return_tensors="pt", truncation=True, padding="max_length", max_length=100) # 512
    inputs = {key: value.to(training_args.device) for key, value in inputs.items() if key != "token_type_ids"}  # token_type_ids 제거
    with torch.no_grad():
        outputs = model(**inputs)  # 하나의 출력만 받음
        predicted_scores = outputs.squeeze().tolist()  # 직접 예측값 추출
        # 예측 결과를 원래 점수 범위(0-100)로 역정규화
        original_predicted_scores = [score * 100 for score in predicted_scores]
        print(f"Segment {i} Predicted Evaluation Scores: {original_predicted_scores}")

/home/viplab/anaconda3/envs/deepfake/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-25 23:00:09.433601: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-25 23:00:09.442689: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732543209.453816 3292519 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732543209.457129 3292519 cuda_blas.cc

Step,Training Loss
10,0.170100
20,0.089900
30,0.074300
40,0.070700
50,0.060500
60,0.063500
70,0.060300
80,0.062300
90,0.051600
100,0.051000



Input Data and Predicted Scores:
      Segment                                          MIDI Data  \
0   Segment 1  time_0.0: note_48_velocity_66; time_0.1: note_...   
1   Segment 2  time_10.0: note_40_velocity_52, note_44_veloci...   
2   Segment 3  time_20.0: note_42_velocity_53, note_50_veloci...   
3   Segment 4  time_30.0: note_51_velocity_58; time_30.1: not...   
4   Segment 5  time_40.0: note_52_velocity_67, note_42_veloci...   
5   Segment 6  time_50.0: note_55_velocity_57, note_43_veloci...   
6   Segment 7  time_60.0: note_47_velocity_67, note_41_veloci...   
7   Segment 8  time_70.0: note_40_velocity_70, note_40_veloci...   
8   Segment 9  time_80.0: note_47_velocity_53; time_80.1: not...   
9  Segment 10  time_90.0: note_49_velocity_60; time_90.1: not...   

                                              Scores  
0  [74.41527430166627, 21.40132424941519, 44.2725...  
1  [73.73804405775856, 18.66351952076304, 55.2340...  
2  [44.24998876753039, 61.68061990148321, 64.5076...